# Soda spark - deprecated

## Setup

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path 
import pandas as pd
from utils.data_generation import FakerProfileDataSnapshot

from pyspark.sql import SparkSession
from  pyspark.sql.functions import input_file_name, split, regexp_extract

In [3]:
from delta import *

In [4]:
from pyspark.sql import SparkSession
from delta import *

builder = SparkSession.builder.appName("data-pipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [9]:
# object for data generation
datagen = FakerProfileDataSnapshot(
    data_dir=str(Path('./data').resolve())
)
# clean landing storage
datagen.delete_batches_in_landing_dir()
print(datagen.bronze_dir)
datagen.delete_bronze_dir()

## Parameters

In [10]:
read_csv_options = {'header': 'true', 'inferSchema': 'true', 'delimiter': ',', 'quote': '"', 'multiLine': 'true'}

In [11]:
location_landing = datagen.landing_dir
location_bronze = datagen.bronze_dir

## Processing

### Generate data - CSV

In [12]:
path_csv = datagen.land_batch(7)
path_csv

'/home/jovyan/work/data/landing/snapshot_2022-09-30.csv'

### Create delta table 

In [13]:
df = spark.read.format('csv').options(**read_csv_options).load(str(path_csv)) \
    .withColumn("_delivery", regexp_extract(input_file_name(), '.*_(.*).csv$', 1))
df.toPandas()

,id,job,company,ssn,website,username,name,address,mail,birthdate,_delivery
0,0,Contractor,"Boone, Gallagher and Scott",508-98-7365,"['https://hall.info/', 'https://www.monroe-haw...",paula86,Misty Phillips,"4978 Chapman Bypass\nSanchezfurt, TN 23177",ana51@yahoo.com,1996-12-21,2022-09-30
1,1,Rural practice surveyor,Osborne PLC,861-51-6071,"['https://kelley.net/', 'http://www.herring-ca...",andreawhite,Alfred Hall,"11070 Wright Creek Apt. 541\nEast Jonathan, TN...",shawn57@yahoo.com,1922-03-06,2022-09-30
2,3,Newspaper journalist,Gordon-Smith,627-52-4610,['http://www.jackson.com/'],matthew14,Kyle Randall,"98150 Jones Way Apt. 251\nJonesside, OK 11215",michael49@hotmail.com,1922-03-12,2022-09-30
3,4,"Programmer, systems",Rodriguez-Williams,303-02-0239,"['https://www.pearson.com/', 'http://carlson.c...",kdavis,Sally Davis,"4052 Sparks Prairie\nJohnville, RI 76389",sanchezandrew@gmail.com,1961-04-26,2022-09-30
4,6,"Surveyor, rural practice",Jackson-Davis,498-01-2910,"['https://evans.info/', 'http://www.choi.com/']",simpsonphillip,Keith Todd,"280 Lindsey Road Apt. 412\nSouth Melissabury, ...",umartin@yahoo.com,1923-10-30,2022-09-30
5,9,"Surveyor, commercial/residential",Kelly-Singh,402-72-6549,"['http://www.english.com/', 'https://wilson.co...",rpeterson,Kelsey Ramsey,"36377 Christensen Fort\nPort Justin, OR 51565",shannonscott@yahoo.com,1987-06-05,2022-09-30
6,12,Secondary school teacher,Hernandez Ltd,334-42-9390,['http://walker.biz/'],harriscarl,Veronica Maldonado,"32860 Yoder Oval\nJeffreyborough, NM 31331",xchavez@hotmail.com,1932-05-02,2022-09-30
7,13,Race relations officer,"Jones, Randall and Cherry",418-26-0136,"['https://turner-hunter.org/', 'http://www.fie...",hbryant,Samuel Mack,"2024 Hodge Radial\nStefaniechester, NY 38154",bvazquez@yahoo.com,2012-05-20,2022-09-30
8,14,"Radiographer, diagnostic",Baldwin Group,412-92-6966,"['https://shah.net/', 'https://www.cohen-white...",gphillips,Anita Price,"97126 Gary Glens Suite 847\nPort Katherine, NM...",toddjessica@yahoo.com,2003-02-07,2022-09-30
9,15,"Geologist, engineering",Fernandez Group,308-98-1651,"['https://morgan.com/', 'https://www.gordon.co...",robert00,Courtney Gonzalez,"34549 Johnson Isle\nDonaldview, WI 19280",reedchristopher@gmail.com,1947-06-19,2022-09-30


In [14]:
df.write.format('delta').option('overwriteSchema', 'true').mode('overwrite').save(location_bronze)

In [15]:
spark.sql('DROP TABLE IF EXISTS snapshot')
spark.sql(f'CREATE TABLE IF NOT EXISTS snapshot USING DELTA LOCATION "{location_bronze}"')

DataFrame[]

In [16]:
spark.sql(f'DESCRIBE HISTORY snapshot').toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2022-08-21 09:41:18.061,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '14', 'numOutputBytes': '628...",None,Apache-Spark/3.2.1 Delta-Lake/2.0.0


### Run soda spark scan

https://docs.soda.io/soda-spark/install-and-use.html#use-soda-spark

In [33]:
from sodaspark import scan

scan_definition = ("""
table_name: snapsho
metric_groups:
- all
tests:
- row_count == 13
columns:
  id:
    valid_format: uuid
    tests:
    - invalid_percentage == 0
    - missing_counts == adfgvadf
""")
scan_result = scan.execute(scan_definition, df)

ERROR:sodasql.scan.test:Test error for "missing_counts == adfgvadf": name 'missing_counts' is not defined


https://github.com/sodadata/soda-spark/blob/main/src/sodaspark/scan.py

In [37]:
sdf_measurements = scan.measurements_to_data_frame(scan_result.measurements)
sdf_measurements.toPandas()

,metric,column_name,value,group_values
0,schema,None,"[{logicalType=number, nullable=true, semanticT...",None
1,row_count,None,14,None
2,values_count,id,14,None
3,valid_count,id,14,None
4,min,id,0,None
...,...,...,...,...
198,duplicate_count,_delivery,1,None
199,uniqueness,_delivery,0.0,None
200,mins,_delivery,[2022-09-30],None
201,maxs,_delivery,[2022-09-30],None


In [35]:
sdf_test_results = scan.test_results_to_data_frame(scan_result.test_results)
sdf_test_results.toPandas()

,test,passed,skipped,values,error,group_values
0,"({""expression"":""row_count == 13""}, test(row_co...",False,False,"{'expression_result': '14', 'row_count': '14'}",None,None
1,"({""column"":""id"",""expression"":""invalid_percenta...",True,False,"{'invalid_percentage': '0.0', 'expression_resu...",None,None
2,"({""column"":""id"",""expression"":""missing_counts =...",False,False,None,net.razorvine.pickle.PythonException: [builtin...,None


In [36]:
sdf_scan_errors = scan.scan_errors_to_data_frame(scan_result.errors)
sdf_scan_errors.toPandas()

,message,exception
0,"Test ""missing_counts == adfgvadf"" failed",net.razorvine.pickle.PythonException: [builtin...
